<a href="https://colab.research.google.com/github/crdsteixeira/OR_project/blob/main/Scheduling_Aircraft_Landings_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the library

In [1]:
!pip install docplex
!pip install cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 640 kB 4.0 MB/s 
  Created wheel for docplex: filename=docplex-2.24.232-py3-none-any.whl size=682306 sha256=76f1a6cf8c7f12d542147865dc5b68807275ddc85900832447cef43be5b4a799
  Stored in directory: /root/.cache/pip/wheels/cd/84/5d/b9c307d9cf361c49d41ddea36761e226bba3afdfd038673dcd
Successfully built docplex
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 1.7 MB/s 


In [2]:
from docplex.cp.model import *
import numpy as np

# Info

The format of these data files is:
number of planes (p), freeze time
for each plane i (i=1,...,p):
   - appearance time, 
   - earliest landing time, 
   - target landing time,
   - latest landing time, 
   - penalty cost per unit of time for landing before target, 
   - penalty cost per unit of time for landing after target

for each plane j (j=1,...p): separation time required after 
                                i lands before j can land


### Sigle runway definitions and variables

![image.png](attachment:image.png)

### Multirunway defintions and variables 

![image.png](attachment:image.png)

## Get data

In [ ]:
def read_datafiles(file):
    with open(file, 'r') as data:
        data_lines = data.readlines()
        number_planes = int(data_lines[0].split()[0])
        freeze_time =  int(data_lines[0].split()[1])
        mixed_data = [line.split() for line in data_lines[1:]]
        
        flight_details = np.empty([0,6],dtype=float)
        separation_time = np.empty([0,number_planes],dtype=float)
        
        flag = 0
       
        for element in mixed_data:
            if flag == 0: # flight details
                flight_details = np.vstack([flight_details, np.array(element)])
                flag = 1
                element_final = []
            else:  # separation_times
                element_final.extend(element)
                if len(element_final) == number_planes:
                    separation_time = np.vstack([separation_time, np.array(element_final)])
                    flag = 0
                
        print(f" number planes: {number_planes}")
        print(f" freeze time: {freeze_time}")
        print(f" mixed data: {mixed_data}")
        print(f" flight details: {flight_details}")
        print(f" separation time: {separation_time}")
    
    return number_planes, freeze_time, flight_details, separation_time


In [ ]:
read_datafiles("./Data/airland1.txt")

 number planes: 10
 freeze time: 10
 mixed data: [['54', '129', '155', '559', '10.00', '10.00'], ['99999', '3', '15', '15', '15', '15', '15', '15'], ['15', '15'], ['120', '195', '258', '744', '10.00', '10.00'], ['3', '99999', '15', '15', '15', '15', '15', '15'], ['15', '15'], ['14', '89', '98', '510', '30.00', '30.00'], ['15', '15', '99999', '8', '8', '8', '8', '8'], ['8', '8'], ['21', '96', '106', '521', '30.00', '30.00'], ['15', '15', '8', '99999', '8', '8', '8', '8'], ['8', '8'], ['35', '110', '123', '555', '30.00', '30.00'], ['15', '15', '8', '8', '99999', '8', '8', '8'], ['8', '8'], ['45', '120', '135', '576', '30.00', '30.00'], ['15', '15', '8', '8', '8', '99999', '8', '8'], ['8', '8'], ['49', '124', '138', '577', '30.00', '30.00'], ['15', '15', '8', '8', '8', '8', '99999', '8'], ['8', '8'], ['51', '126', '140', '573', '30.00', '30.00'], ['15', '15', '8', '8', '8', '8', '8', '99999'], ['8', '8'], ['60', '135', '150', '591', '30.00', '30.00'], ['15', '15', '8', '8', '8', '8', '8',

# Set up the prescriptive model

In [3]:
def MIP_model(fname,R):
    from docplex.mp.model import Model
    mdl = Model("Aircraft Landing - Multi Runaway Static Case")
    
    P, freeze_time, flight_details, separation_time = read_datafiles(fname)

    #Creating a CPLEX model
    #model=Model("Aircraft Landing Schedule")
    
    # The first column relates to the actual appearence time of the plane so will not be taken into account for our decision variables

    E = flight_details[:,1]  #earliest landing time,
    T = flight_details[:,2]  #target landing time,
    L = flight_details[:,3]  #latest landing time,
    g = flight_details[:,4]  #penalty cost per unit of time for landing before target,
    h = flight_details[:,5]  #penalty cost per unit of time for landing after target
    r = max(flight_details[:,3]) - min(flight_details[:,1]) #landing range time

    
    g_cost = [] #penalty cost per unit time for landing before the target time
    h_cost = [] #penalty cost per unit time for landing after the target time
    for i in np.arange(P):
        g_cost[i] = flight_details[i,4]
        h_cost[i] = flight_details[i,5]

    delta_dict = []
    z_dict = []
    S_dict = [] # the required separation time (>= 0) between plane i landing and plane j landing 
    ij = []
    for i in np.arange(P):
        for j in np.arange(P):
          if flight_details[i,0] < flight_details[j,0]:
            delta_dict[i,j] = 1 #aqui os gajos dizem que todos os avioes j chegam antes do i mas isso nao faz muito sentido, temos de fazer um check do appearence time ( foi o que tentei fazer)
          else:
            delta_dict[i,j] = 0

          z_dict[i,j] = 0
          S_dict[i,j] = separation_time[i,j]
          ij[i] = ((i,j))

    ir = []
    y_dict = {}
    for i in np.arange(P):
        for r in np.arange(R):
            y_dict[i,r]=0        
            ir.append((i,r))

    #Defining decision variables

    alpha   = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="alpha") # how soon a plane lands before target time
    beta    = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="beta") # how soon a plane lands after target time
    x       = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="x") # landing time for a plane
    delta   = mdl.binary_var_dict(ij,lb=0,ub=1, name="delta") # binary variable: 1 if plane i lands before plane j, 0 elsewhere
    y       = mdl.binary_var_dict(ir,lb=0,ub=1, name="y") # NAO SEI O QUE É
    z       = mdl.binary_var_dict(ij,lb=0,ub=1, name="z") # NAO SEI O QUE É



    #Adding constraints
    mdl.add_constraints(x[i]>=E[i-1] for i in np.arange(P))                                                                       # Landing time of plane i must be later than the earliest landing time
    mdl.add_constraints(x[i]<=L[i-1] for i in np.arange(P))                                                                       # Landing time of plane i must be earlier than the before latest landing time
    mdl.add_constraints(delta[i,j]+delta[j,i]==1 for i in np.arange(P) for j in np.arange(P) if j!=i)                        # Either plane i lands before plane j or plane j lands before plane i, but not both
    mdl.add_constraints(alpha[i]>=T[i-1]-x[i] for i in np.arange(P))                                                              # How soon plane i lands before T[i] must be larger than T[i] - x[i]
    mdl.add_constraints(beta[i]>=x[i]-T[i-1] for i in np.arange(P))                                                               # How soon plane i lands after T[i] must be larger than x[i] - T[i]
    mdl.add_constraints(x[j]-x[i]>=S_dict[i,j]*z[j,i] - (delta[j,i])*r for i in np.arange(P) for j in np.arange(P) if j!=i)   # Separation time between plane i and plane j must be respected
    mdl.add_constraints(z[i,j]==z[j,i] for i in np.arange(P) for j in np.arange(P) if j>i)                                   # If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
    mdl.add_constraints(mdl.sum(y[i,r] for r in np.arange(R))==1 for i in np.arange(P))                                       # Plane i can only land in 1 runaway
    mdl.add_constraints(z[i,j]>=y[i,r]+y[j,r]-1 for r in np.arange(R) for j in np.arange(P) for i in np.arange(P) if j>i) # If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 


    total_cost = mdl.sum(beta[i] * h[i-1] + alpha[i] * g[i-1] for i in np.arange(P))

    mdl.minimize(total_cost)
    
    mdl.print_information()

    msol = mdl.solve()
    assert msol is not None, "model can't solve"

    return mdl

## Create CPO Model

## Define the decision variables

## Define constraints

## Define the objective

## Solve with decision optimization